In [34]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.ops import unary_union

## Open Geojson

In [2]:
czechia_gdf = gpd.read_file("data/geojson/Okresy_LAU_1_multi_20250701.geojson")

In [3]:
czechia_gdf["naz_okres"] = czechia_gdf["naz_okres"].str.lower()

## Get Open Population data

In [4]:
pop_2015_df = pd.read_excel("data/DEM13_2015.xlsx", sheet_name=None, engine="openpyxl")

/mnt/f/Github/VineMapper/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
pop_2015_df = (
    pop_2015_df["DATA"][["Unnamed: 1", "Unnamed: 2"]]
    .dropna()
    .reset_index(drop=True)[1:]
)
pop_2015_df = pop_2015_df.rename(
    columns={"Unnamed: 1": "naz_okres", "Unnamed: 2": "2015"}
)
pop_2015_df["2015"] = pop_2015_df["2015"].astype(int)

In [6]:
pop_2025_df = pd.read_excel("data/DEM13_2025.xlsx", sheet_name=None, engine="openpyxl")

/mnt/f/Github/VineMapper/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
pop_2025_df = (
    pop_2025_df["DATA"][["Unnamed: 1", "Unnamed: 2"]]
    .dropna()
    .reset_index(drop=True)[1:]
)
pop_2025_df = pop_2025_df.rename(
    columns={"Unnamed: 1": "naz_okres", "Unnamed: 2": "2025"}
)
pop_2025_df["2025"] = pop_2025_df["2025"].astype(int)

In [8]:
pop_df = pop_2015_df.merge(pop_2025_df, on="naz_okres", how="inner")

In [9]:
pop_df["DIFF"] = pop_df["2025"] - pop_df["2015"]
pop_df["PERCENT"] = ((pop_df["DIFF"] / pop_df["2015"]) * 100.0).round(decimals=1)

In [10]:
pop_df[pop_df["naz_okres"] == "Czech Republic"]

,naz_okres,2015,2025,DIFF,PERCENT
0,Czech Republic,10538275,10909500,371225,3.5


In [19]:
pop_df = pop_df.replace("praha, the capital city", "praha")

## Merge data and export

In [20]:
pop_df["naz_okres"] = pop_df["naz_okres"].str.lower()

In [21]:
pop_gdf = czechia_gdf[["naz_okres", "geometry"]].merge(
    pop_df, on="naz_okres", how="left"
)

In [23]:
pop_gdf = pop_gdf.to_crs(5514)
pop_gdf.to_file("data/Population_Change_Czechia_2015_2025.gpkg")

In [32]:
czechia_gdf = czechia_gdf.to_crs(5514)

In [37]:
czechia_gdf.dissolve(by="naz_kraj")[["geometry"]].to_file("data/czech_regions.gpkg")
merged = unary_union(czechia_gdf.geometry)
outline = merged.boundary
outline_gdf = gpd.GeoDataFrame(geometry=[outline], crs=czechia_gdf.crs)
outline_gdf.to_file("data/czechia_boundary.gpkg")

In [27]:
pop_gdf[["naz_okres", "PERCENT", "DIFF"]].sort_values("PERCENT", ascending=False).head()

,naz_okres,PERCENT,DIFF
8,praha-východ,24.7,40545
9,praha-západ,20.5,27569
22,plzeň-jih,15.3,9502
1,beroun,15.0,13390
7,nymburk,11.3,10949


In [28]:
pop_gdf[["naz_okres", "PERCENT", "DIFF"]].sort_values("PERCENT", ascending=True).head()

,naz_okres,PERCENT,DIFF
19,domažlice,-8.4,-5102
61,jeseník,-7.8,-3092
72,karviná,-6.8,-17526
70,bruntál,-6.6,-6285
28,sokolov,-5.7,-5166


In [29]:
pop_gdf[["naz_okres", "PERCENT", "DIFF"]].sort_values("DIFF", ascending=False).head()

,naz_okres,PERCENT,DIFF
76,praha,11.0,138801
8,praha-východ,24.7,40545
9,praha-západ,20.5,27569
55,brno-město,6.7,25299
56,brno-venkov,10.1,21466


In [30]:
pop_gdf[["naz_okres", "PERCENT", "DIFF"]].sort_values("DIFF", ascending=True).head()

,naz_okres,PERCENT,DIFF
72,karviná,-6.8,-17526
75,ostrava-město,-2.9,-9485
33,most,-5.6,-6321
70,bruntál,-6.6,-6285
29,děčín,-4.5,-5909
